In [48]:
##importando pacotes
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import csv


Matplotlib is building the font cache; this may take a moment.


In [2]:
##chamando diretório
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Dados para subir\\Dados Brutos\\INEP\\Adequação Formação Docente')

In [24]:
#chamando a tabela com todos os anos de Adequação da Formação Docente 
df = pd.read_csv('Tabela final.csv',  sep=";", encoding='UTF8')
df

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_11668\1752520713.py:2: DtypeWarning: Columns (9,19,20,21,22,23,34,35,36,37,38,44,45,46,47,48,49,50,51,52,53,54) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Tabela final.csv',  sep=";", encoding='UTF8')


,Ano,Região,UF,Código do Município,Nome do Município,Código da Escola,Nome da Escola,Localização,Dependência Administrativa,EI_grupo1,...,EF_iniciais_grupo2,EF_iniciais_grupo3,EF_iniciais_grupo4,EF_iniciais_grupo5,EJA_EF_grupo1\n,EJA_EF_grupo2\n,EJA_EF_grupo3\n,EJA_EF_grupo4\n,EJA_EF_grupo5\n,EI_grupo1\n
0,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11022558,EIEEF HAP BITT TUPARI,Rural,Estadual,--,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024275,CEEJA LUIZ VAZ DE CAMOES,Urbana,Estadual,--,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024291,EMMEF 07 DE SETEMBRO,Rural,Municipal,--,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024313,EMMEF AFONSO AURELIO PORTO,Rural,Municipal,--,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013,Norte,RO,1100015,ALTA FLORESTA D OESTE,11024372,EMEIEF ANA NERY,Rural,Municipal,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1802862,2022,Centro-Oeste,DF,5300108,Brasília,53084020,ESC PEQUENO ENCANTO,Urbana,Privada,100,...,14.3,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1802863,2022,Centro-Oeste,DF,5300108,Brasília,53084039,COL ANCHIETA,Urbana,Privada,26.8,...,0,0,0,6.7,NaN,NaN,NaN,NaN,NaN,NaN
1802864,2022,Centro-Oeste,DF,5300108,Brasília,53084055,COL MAPA,Urbana,Privada,100,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1802865,2022,Centro-Oeste,DF,5300108,Brasília,53085000,COBIAN - COL BIANGULO,Urbana,Privada,75.5,...,0,4.2,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
#vendo valores inclusos na coluna "'Dependência Administrativa'"
df['Dependência Administrativa'].unique()

array(['Estadual', 'Municipal', 'Privada', 'Federal'], dtype=object)

In [32]:
## Pegando uma parte do df para trabalhar apenas com os grupos do Ensino Fundamental (Sigla EF) e Ensino Médio (Sigla EM). 
recorte = df[['Ano','Dependência Administrativa','EF_total_grupo1', 'EF_finais_grupo1','EM_grupo1',]]
recorte

,Ano,Dependência Administrativa,EF_total_grupo1,EF_finais_grupo1,EM_grupo1
0,2013,Estadual,0,0,--
1,2013,Estadual,--,--,--
2,2013,Municipal,0,0,--
3,2013,Municipal,0,0,--
4,2013,Municipal,28.9,22.2,--
...,...,...,...,...,...
1802862,2022,Privada,85.7,--,--
1802863,2022,Privada,84.4,63.2,--
1802864,2022,Privada,84.2,70,92.1
1802865,2022,Privada,95.8,--,--


In [33]:
#O grupo 1 são docentes com formação superior de licenciatura (ou bacharelado com complementação pedagógica) na mesma área da disciplina que leciona.
#Existem valores como "--" que impossibilitam algumas funções futuras, então é preciso tranformá-la em valor nulo (NaN)
recorte['EF_total_grupo1'] = np.where(recorte['EF_total_grupo1']=="--", np.nan,recorte['EF_total_grupo1'])
recorte['EM_grupo1'] = np.where(recorte['EM_grupo1']=="--", np.nan,recorte['EM_grupo1'])
recorte['EF_finais_grupo1'] = np.where(recorte['EF_finais_grupo1']=="--", np.nan,recorte['EF_finais_grupo1'])


C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_11668\3368133202.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recorte['EF_total_grupo1'] = np.where(recorte['EF_total_grupo1']=="--", np.nan,recorte['EF_total_grupo1'])
C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_11668\3368133202.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recorte['EM_grupo1'] = np.where(recorte['EM_grupo1']=="--", np.nan,recorte['EM_grupo1'])
C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_1166

In [34]:

recorte


,Ano,Dependência Administrativa,EF_total_grupo1,EF_finais_grupo1,EM_grupo1
0,2013,Estadual,0,0,NaN
1,2013,Estadual,NaN,NaN,NaN
2,2013,Municipal,0,0,NaN
3,2013,Municipal,0,0,NaN
4,2013,Municipal,28.9,22.2,NaN
...,...,...,...,...,...
1802862,2022,Privada,85.7,NaN,NaN
1802863,2022,Privada,84.4,63.2,NaN
1802864,2022,Privada,84.2,70,92.1
1802865,2022,Privada,95.8,NaN,NaN


In [35]:
#Na coluna EM_grupo1, transformando em float os valores que estão como objetos. 
recorte['EM_grupo1'] = pd.to_numeric(recorte['EM_grupo1'])
recorte['EF_total_grupo1'] = pd.to_numeric(recorte['EF_total_grupo1'])
recorte['EF_finais_grupo1'] = pd.to_numeric(recorte['EF_finais_grupo1']) 
recorte

C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_11668\583126699.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recorte['EM_grupo1'] = pd.to_numeric(recorte['EM_grupo1'])
C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_11668\583126699.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recorte['EF_total_grupo1'] = pd.to_numeric(recorte['EF_total_grupo1'])
C:\Users\ana.sales_republica\AppData\Local\Temp\ipykernel_11668\583126699.py:4: SettingWithCopyWarning: 
A value is trying to be s

,Ano,Dependência Administrativa,EF_total_grupo1,EF_finais_grupo1,EM_grupo1
0,2013,Estadual,0.0,0.0,NaN
1,2013,Estadual,NaN,NaN,NaN
2,2013,Municipal,0.0,0.0,NaN
3,2013,Municipal,0.0,0.0,NaN
4,2013,Municipal,28.9,22.2,NaN
...,...,...,...,...,...
1802862,2022,Privada,85.7,NaN,NaN
1802863,2022,Privada,84.4,63.2,NaN
1802864,2022,Privada,84.2,70.0,92.1
1802865,2022,Privada,95.8,NaN,NaN


In [36]:
recorte.dtypes

Ano                             int64
Dependência Administrativa     object
EF_total_grupo1               float64
EF_finais_grupo1              float64
EM_grupo1                     float64
dtype: object

In [58]:
recorte['EF_total_grupo1'].mean()


45.68539500038222

In [45]:
#Fazendo média de público e privado para grupo 1 do EF por ano, de 2013 até 2022. (em pivot table para melhor visualização)
media_ef_total_grupo1 = pd.pivot_table(recorte, values='EF_total_grupo1', index='Ano', columns='Dependência Administrativa', aggfunc='mean')

#Fazendo média de público e privado para grupo 1 dos anos finais do EF por ano, de 2013 até 2022. (em pivot table para melhor visualização)

media_ef_finais_grupo1 = pd.pivot_table(recorte, values='EF_total_grupo1', index='Ano', columns='Dependência Administrativa', aggfunc='mean')

#Fazendo média de público e privado para grupo 1 do EM por ano, de 2013 até 2022. (em pivot table para melhor visualização)
media_em_grupo1 = pd.pivot_table(recorte, values='EM_grupo1', index='Ano', columns='Dependência Administrativa', aggfunc='mean')

# Mostrando as médias do Grupo 1 do Ensino Médio
media_em_grupo1

Dependência Administrativa,Estadual,Federal,Municipal,Privada
Ano,,,,
2013,54.810695,73.954595,49.107602,57.154067
2014,56.416598,73.837500,49.681356,58.348335
2015,56.163458,73.303888,50.682156,57.986159
2016,57.452250,74.048039,51.544400,57.999879
2017,58.438179,74.434369,50.544672,58.324788
2018,59.266832,74.778571,55.350526,59.257115
2019,61.716645,70.579966,56.313514,60.678246
2020,63.470503,74.806511,57.548066,63.391169
2021,65.153971,74.734007,57.181283,66.206084


In [46]:
# Mostrando as médias do Grupo 1 dos anos finais do Ensino Fundamental
media_ef_finais_grupo1

Dependência Administrativa,Estadual,Federal,Municipal,Privada
Ano,,,,
2013,56.322578,81.406383,30.733807,41.828641
2014,58.314864,80.155319,33.232037,43.153850
2015,59.067289,81.206522,34.746721,43.379875
2016,60.559869,81.104255,36.702942,44.009541
2017,61.213824,81.672340,38.549373,44.792639
2018,63.543079,83.676596,40.767312,46.532918
2019,65.930942,74.170213,44.173186,51.050821
2020,67.569278,78.006522,46.926230,53.570913
2021,68.059584,78.882979,49.145060,55.709473


In [47]:
# Mostrando as médias do Grupo 1 dos anos finais + iniciais do Ensino Fundamental
media_ef_total_grupo1

Dependência Administrativa,Estadual,Federal,Municipal,Privada
Ano,,,,
2013,56.322578,81.406383,30.733807,41.828641
2014,58.314864,80.155319,33.232037,43.153850
2015,59.067289,81.206522,34.746721,43.379875
2016,60.559869,81.104255,36.702942,44.009541
2017,61.213824,81.672340,38.549373,44.792639
2018,63.543079,83.676596,40.767312,46.532918
2019,65.930942,74.170213,44.173186,51.050821
2020,67.569278,78.006522,46.926230,53.570913
2021,68.059584,78.882979,49.145060,55.709473


In [ ]:
## gráfico de linha mostrando ao longo do tempo para as 4 redes
Rede = ['Privada', 'Estadual', 'Federal', 'Municipal']
valores = [105235, 107697, 110256, 109236, 108859, 109986]


In [ ]:
#Quantas escolas foram observadas em cada grupo?
